In [1]:
from create_fashion_items_image import create_cw_image, create_fashion_items_image
from embedding_word import get_embedding_word
from fashion_class.FashionItem import FashionItem
from fashion_class.ImageStruct import ImageStruct
import sys
from search_items import search_items_from_caption_embedding

In [2]:
print('データセット初期化中')
# データセットを初期化
annotation_file, tensor_file = \
      './data/anotation_new.csv', './data/resnet50_tensor.pt'
dataset = ImageStruct(annotation_file, tensor_file, init_item_length=200)
print('データセット初期化終了')

データセット初期化中
【                                                                                                    】

In [ ]:
import random
# ユーザに表示して、気に入ったアイテムをいくつか選んでもらう
category_list = ["tops", "bottoms", "shoes"]
user_like_items = {
    'tops': [],
    'bottoms': [],
    'shoes': []
}
user_dislike_items = {
    'tops': [],
    'bottoms': [],
    'shoes': [],
}

item_list = {
    'tops': dataset.tops,
    'bottoms':  dataset.bottoms,
    'shoes':  dataset.shoes,
}

for c in category_list:
    fashion_items = random.sample(item_list[c], 10)
    image = create_fashion_items_image(fashion_items)
    image.show()
    # favorite_item_set = list(map(int, input(f'気に入った {c} アイテムのidをスペース区切りで入力してください').split(',')))
    favorite_item_set = [1,2,3]
    for i in favorite_item_set:
        user_like_items[c].append(fashion_items[i-1])

for c in category_list:
    
    fashion_items = random.sample(item_list[c], 10)
    image = create_fashion_items_image(fashion_items)
    image.show()
    # favorite_item_set = list(map(int, input(f'好きではない {c} アイテムのidをスペース区切りで入力してください').split(',')))
    favorite_item_set = [1,2,3]
    for i in favorite_item_set:
        user_dislike_items[c].append(fashion_items[i-1]) 

: 

In [ ]:
import importlib

import fashion_class
from fashion_class import CapsuleWardrobeClass
import CreateCW
importlib.reload(CreateCW)
importlib.reload(fashion_class)
importlib.reload(fashion_class.CapsuleWardrobeClass)
from fashion_class.CapsuleWardrobeClass import CapsuleWardrobe
importlib.reload(CreateCW)
from fashion_class.FashionItem import FashionItem
import gc

from CreateCW import change_item_recommandation, create_cw, search_alternate_item

: 

In [ ]:

# CWを構築
print("与えられた情報をもとにCWを構築しています。")

cw = create_cw(
    {
    'tops': [],
    'bottoms': [],
    'shoes': [],
}, dataset)
print("生成されたcwがこちらです")
# original_cw_score = cw.get_score()
# q = cw.get_score()
# print("scores", original_cw_score, q)
# 画像を表示
while True:
    original_cw_score = cw.get_score()
    gc.collect()
    image = create_cw_image(cw)
    image.show()
    item = input("出来上がったcwに対して、気に入らないitemを1つ tops:2 bottoms:3 のように指定してください。CWが完成した場合はqと入力してください > ")
    if item == 'q':
        break
    kind, index = item.split(':')
    index = int(index)
    if kind != 'tops' and kind != 'bottoms' and kind != 'shoes':
        raise Exception('ValueError')
        # continue
    new_item = search_alternate_item(cw, kind, index-1, dataset, user_like_items, user_dislike_items)
    # 1枚多く計算する
    # 既存のアイテムは外さない
    required_items = {
        'tops': cw.get_tops(),
        'bottoms':cw.get_bottoms(),
        'shoes':cw.get_shoes()
    }
    new_item_image = new_item.create_image()
    new_item_image.show()

    # 既存のアイテムのお気に入りの入れ替え
    required_items[kind][index-1] = new_item
    cw_replacement = CapsuleWardrobe(initial_items=required_items)
    cw_replacement_score = cw_replacement.get_score()

    alternate_cw = create_cw(required_items, dataset, max_length=cw.max_length + 1)
    new_items = {
        'tops': alternate_cw.get_tops()[-1],
        'bottoms': alternate_cw.get_bottoms()[-1],
        'shoes': alternate_cw.get_shoes()[-1]
    }
    image = create_cw_image(alternate_cw)
    image.show()
    recommended_change = change_item_recommandation(alternate_cw)
    for (r, i) in recommended_change.items():
        if r == kind or i+1 == 5:
            continue
        can_change = input(f'{r}の{i+1}番目を、新しい推薦アイテムと変えるとより良いCWになりますが、変えますか？ yes/no')
        if can_change == 'yes':
            required_items[r][i] = new_items[r]
    cw = CapsuleWardrobe(initial_items=required_items)
    cw_changed_score = cw.get_score()
    print(f'start: {original_cw_score} replacement: {cw_replacement_score} changed: {cw_changed_score}')

: 

In [ ]:
image = create_cw_image(cw)
image.show()

: 

In [ ]:
[i.item_id for i in cw.get_bottoms()]

: 

In [ ]:
[i.item_id for i in cw.get_shoes()]

: 

In [ ]:
[i.item_id for i in cw.get_tops()]

: 